In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [134]:
a=gen.load_zinc(n_samples=10)

In [135]:
a

[Graph(num_nodes=29, num_edges=64,
       ndata_schemes={'attr': Scheme(shape=(28,), dtype=torch.float32), 'feat': Scheme(shape=(), dtype=torch.int64)}
       edata_schemes={'attr': Scheme(shape=(4,), dtype=torch.float32), 'feat': Scheme(shape=(), dtype=torch.int64)}),
 Graph(num_nodes=26, num_edges=56,
       ndata_schemes={'attr': Scheme(shape=(28,), dtype=torch.float32), 'feat': Scheme(shape=(), dtype=torch.int64)}
       edata_schemes={'attr': Scheme(shape=(4,), dtype=torch.float32), 'feat': Scheme(shape=(), dtype=torch.int64)}),
 Graph(num_nodes=16, num_edges=34,
       ndata_schemes={'attr': Scheme(shape=(28,), dtype=torch.float32), 'feat': Scheme(shape=(), dtype=torch.int64)}
       edata_schemes={'attr': Scheme(shape=(4,), dtype=torch.float32), 'feat': Scheme(shape=(), dtype=torch.int64)}),
 Graph(num_nodes=27, num_edges=60,
       ndata_schemes={'attr': Scheme(shape=(28,), dtype=torch.float32), 'feat': Scheme(shape=(), dtype=torch.int64)}
       edata_schemes={'attr': Scheme(s

In [138]:
nx_a = [dgl.to_networkx(a.cpu())  for a in a]

In [140]:
nx_a[0]

In [141]:

lobsters = a
grids = a

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

grids = [ dgl.from_networkx(g).to(device) for g in grids] # Convert graphs to DGL from NetworkX
lobsters = [ dgl.from_networkx(g).to(device) for g in lobsters] # Convert graphs to DGL from NetworkX

"""
grids = [ dgl.from_networkx(g).to(device) for g in grids] # Convert graphs to DGL from NetworkX
lobsters = [ dgl.from_networkx(g).to(device) for g in lobsters] # Convert graphs to DGL from NetworkX

"""

AttributeError: 'DGLHeteroGraph' object has no attribute 'is_directed'

In [121]:
print(grids[0],"\n", lobsters[0])

Graph(num_nodes=327, num_edges=1798,
      ndata_schemes={}
      edata_schemes={}) 
 Graph(num_nodes=327, num_edges=1798,
      ndata_schemes={}
      edata_schemes={})


In [122]:
# Compute all GNN-based metrics at once
from evaluation.evaluator import Evaluator
evaluator = Evaluator(device=device)
evaluator.evaluate_all(generated_dataset=grids, reference_dataset=lobsters)

orthogonal


{'activations_time': 0.0359041690826416,
 'fid': -7.3418430730498585e-06,
 'fid_time': 0.10272598266601562,
 'kid': -0.8292633,
 'kid_time': 0.051862239837646484,
 'precision': 1.0,
 'recall': 1.0,
 'f1_pr': 1.00001,
 'precision_time': 0.07978701591491699,
 'recall_time': 0.07978701591491699,
 'f1_pr_time': 0.07978701591491699,
 'density': 1.2000000000000002,
 'coverage': 1.0,
 'f1_dc': 1.090919173552968,
 'density_time': 0.05385470390319824,
 'coverage_time': 0.05385470390319824,
 'f1_dc_time': 0.05385470390319824,
 'mmd_rbf': 0,
 'mmd_rbf_time': 0.05984139442443848,
 'mmd_linear': 0.0,
 'mmd_linear_time': 0.0359041690826416}

In [123]:
# Alternatively, compute a single GNN-based metric. See evaluation/gin_evaluation.py for other metrics.
from evaluation.gin_evaluation import load_feature_extractor, MMDEvaluation

# Can tweak GIN hyperparameters, however defaults are set to our recommendations
gin = load_feature_extractor(device=device)
# Can tweak hyperparameters of MMD RBF, however defaults are set to our recommendations
mmd_eval = MMDEvaluation(gin)
result, time = mmd_eval.evaluate(generated_dataset=grids, reference_dataset=lobsters)
print('result: {}, time to compute: {:.3f}s'.format(result, time))

orthogonal
result: {'mmd_rbf': 0}, time to compute: 0.063s


In [124]:
# Evaluation incorporating custom node/edge features.
# If no node features are provided, the default is degree features
# expressed as an integer.

node_feat_loc = 'feat'
edge_feat_loc = 'feat'
node_feat_dim = 10
edge_feat_dim = 5

# Create random node/edge features
for g in grids:
    num_nodes = g.number_of_nodes()
    g.ndata[node_feat_loc] = torch.randn(num_nodes, node_feat_dim).to(g.device)
    num_edges = g.number_of_edges()
    g.edata[edge_feat_loc] = torch.randn(num_edges, edge_feat_dim).to(g.device)
    
for g in lobsters:
    num_nodes = g.number_of_nodes()
    g.ndata[node_feat_loc] = torch.randn(num_nodes, node_feat_dim).to(g.device)
    num_edges = g.number_of_edges()
    g.edata[edge_feat_loc] = torch.randn(num_edges, edge_feat_dim).to(g.device)

# Load GIN
gin = load_feature_extractor(
    device=device, input_dim=node_feat_dim, edge_feat_dim=edge_feat_dim,
    node_feat_loc=node_feat_loc, edge_feat_loc=edge_feat_loc)

mmd_eval = MMDEvaluation(gin)
result, time = mmd_eval.evaluate(generated_dataset=lobsters, reference_dataset=grids)
print('result: {}, time to compute: {:.3f}s'.format(result, time))

orthogonal
result: {'mmd_rbf': 0.20000000298023224}, time to compute: 0.058s


In [125]:
# To quickly demonstrate sensitivity to node/edge features, let's create
# a copy of the grid graphs and slightly alter the node/edge feat distributions
import copy

result, time = mmd_eval.evaluate(generated_dataset=grids, reference_dataset=grids)
print('MMD RBF for grids with itself: {}'.format(result['mmd_rbf']))

grids_copy = [copy.deepcopy(g) for g in grids]
for g in grids_copy:
    g.ndata[node_feat_loc] *= 1.15
    g.edata[edge_feat_loc] *= 1.15
    
result, time = mmd_eval.evaluate(reference_dataset=grids, generated_dataset=grids_copy)
print('MMD RBF after altering node/edge feat. distributions: {}'.format(result['mmd_rbf']))

MMD RBF for grids with itself: 0
MMD RBF after altering node/edge feat. distributions: 0.20000000298023224
